# Aprendizagem de Máquina I

## Hugo Tremonte de Carvalho

#### hugo@dme.ufrj.br

O objetivo dessa atividade é resolver um problema de regressão linear em uma base de dados real. Mais especificamente, o objetivo é trabalhar sobre [esta base](https://archive.ics.uci.edu/dataset/464/superconductivty+data), do repositório da Universidade da Califórnia, que relaciona a temperatura crítica de supercondutores com estatísticas-resumo de certas propriedades físico-químicas desses materiais. Para mais detalhes, veja [esse artigo](https://arxiv.org/pdf/1803.10260).

*Obs.: A temperatura crítica de um supercondutor é a temperatura abaixo da qual o material perde totalmente sua resistência elétrica e se torna um supercondutor, ou seja, passa a conduzir corrente elétrica com resistência nula e a expelir campos magnéticos de seu interior.*

Esse conjunto de dados consiste de $n = 21.263$ observações e um total de $p = 81$ atributos. Note que ainda não estamos em um cenário tão desfavorável de modo que tenhamos $p \approx n$. A temperatura crítica do supercondutor, representada na coluna de nome `critical_temp`, está medida em Kelvin (K = °C - 273,15), e é tal quantidade que deve ser prevista por você com base nos $p = 81$ atributos.

Portanto, a nossa missão para esta atividade é utilizar as técnicas de regressão que estudamos até o momento para encontrar uma boa forma de prever a variável resposta a partir dos atributos. Para isso, siga o roteiro abaixo:
* Faça uma análise exploratória para entneder como os seus atributos se comportam, se há multicolinearidade, se algum deles é bastante correlacionado com a resposta, etc.
* Note que alguns atributos têm uma ordem de grandeza bem discrepante, o que pode ser numericamente problemático. Iremos tratar disso na terceira parte dessa aula.
* Separe o seu conjunto de dados em treinamento e teste
* Faça ajustes dos modelos de regressão que aprendemos, implementando alguma busca por validação cruzada no conjunto de treinamento para encontrar hiperparâmetros ótimos, quando for pertinente.
* Avalie o desempenho do seu modelo no conjunto de teste, e reporte os seus resultados.

In [7]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet

from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_squared_error as MSE
from sklearn.metrics import mean_absolute_error as MAE
from sklearn.metrics import r2_score as r2

from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor, BaggingRegressor
from sklearn.tree import DecisionTreeRegressor

## Leitura dos dados e análise exploratória

In [ ]:
df = pd.read_csv("https://raw.githubusercontent.com/HugoCarvalhoUFRJ/ap-maq/refs/heads/main/materiais-didaticos/superconductivity.csv", sep = ',')
print(df.shape)
df.head()

In [ ]:
# Verificando se há dados faltantes
df.isnull().sum()

In [ ]:
# Verificando se há linhas duplicadas
df.duplicated().sum()

In [ ]:
# Removendo linhas duplicadas
# https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html
# ATENÇÃO: ESTAMOS SOBRESCREVENDO A VARIÁVEI df

df = df.drop_duplicates()
print(df.shape)

In [ ]:
# Vendo se está tudo em ordem
df

In [ ]:
# Correlação LINEAR entre os atributos e a resposta?

plt.figure(figsize=(15, 15))

mask = np.triu(np.ones_like(df.corr(), dtype=bool))
sns.heatmap(df.corr(method = 'pearson'), annot=False, mask=mask, vmin=-1, vmax=1, cmap = "vlag")
plt.title('Correlações LINEARES entre os atributos e a resposta')
plt.show()

In [ ]:
# Correlação MAIS GERAL entre os atributos?
# https://en.wikipedia.org/wiki/Spearman%27s_rank_correlation_coefficient

plt.figure(figsize=(15, 15))

mask = np.triu(np.ones_like(df.corr(), dtype=bool))
sns.heatmap(df.corr(method = 'spearman'), annot=False, mask=mask, vmin=-1, vmax=1, cmap = "vlag")
plt.title('Correlações MAIS GERAIS entre os atributos')
plt.show()

In [ ]:
# Visualizando de outra forma, focando na variável resposta

plt.figure(figsize = (15,5))
plt.stem(df.corr(method = 'pearson')['critical_temp'], label = 'Pearson', markerfmt = '.b')
plt.stem(df.corr(method = 'spearman')['critical_temp'], label = 'Spearman', markerfmt = '.r')
plt.xticks(range(len(list(df.columns))), list(df.columns), rotation = 'vertical')
plt.legend()

plt.show()

In [ ]:
# Gráficos de dispersão contra a variável resposta

n_cols = 5
n_features = df.shape[1]
n_rows = -(-n_features // n_cols)

fig, axes = plt.subplots(n_rows, n_cols, figsize=(20, 4 * n_rows))
axes = axes.flatten()

for i, col in enumerate(df.columns):
    axes[i].scatter(df[col], df['critical_temp'], alpha=0.5, s=5)
    axes[i].set_title(col)
    axes[i].set_xlabel(col)
    axes[i].set_ylabel("critical_temp")

# Se sobrar espaço na grade, remover eixos extras
for j in range(i+1, len(axes)):
    fig.delaxes(axes[j])

plt.tight_layout()
plt.show()

## Parte 1

Valide o desempenho de modelos de regressão linear e KNN. Mais especificamente, use o conjunto de teste para treinar modelos de regressão linear com e sem penalização (encontrando hiperparâmetros ótimos por validação cruzada) e um modelo de KNN (também encontrando hiperparâmetros ótimos por validação cruzada), e meça o seu desempenho no conjunto de teste.

## Parte 2

Valide o desempenho de modelos de regressão baseados em árvores e o bagging para a regressão linear. Mais especificamente, use o conjunto de teste para treinar modelos de regressão baseados em árvores e o bagging para a regressão linear (encontrando hiperparâmetros ótimos por validação cruzada) e meça o seu desempenho no conjunto de teste.

## Parte 3

Refaça as partes 1 e 2, utilizando uma `Pipeline` para acoplar um `StandardScaler` a cada um dos modelos de regressão empregados.